In [167]:
import sqlite3 as sql
import pandas as pd

In [167]:
conn = sql.connect("data/database.sqlite")

In [167]:
matches = pd.read_sql("""SELECT Match.id, 
                                        Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season, 
                                        stage, 
                                        date,
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        home_team_goal, 
                                        away_team_goal
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                ORDER by date;""", conn)

# Some easy logic to see who won

matches[['home_team_win', 'away_team_win', 'draw']] = pd.get_dummies(matches.apply(lambda row: 0 if row.home_team_goal > row.away_team_goal else 1 if row.home_team_goal < row.away_team_goal else 2, axis=1))
matches['home_team_points'] = matches.apply(lambda row: 3 if row.home_team_win else 0 if row.away_team_win else 1, axis=1)
matches['away_team_points'] = matches.apply(lambda row: 0 if row.home_team_win else 3 if row.away_team_win else 1, axis=1)

# Melt so we can retrieve stats by team

matches_melt = matches.merge(matches.melt(value_vars=['home_team', 'away_team'], var_name='home_away', value_name='team', ignore_index=False), left_index=True, right_index=True)
matches_melt['other_team'] = matches_melt.apply(lambda row: row.away_team if row.home_away == 'home_team' else row.home_team, axis=1)
matches_melt[['win', 'loss']] = pd.get_dummies(matches_melt.apply(lambda row: 0 if ((row.home_away == 'home_team') & row.home_team_win) | ((row.home_away == 'away_team') & row.away_team_win) else None if row.draw else 1, axis=1))
matches_melt['points'] = matches_melt.apply(lambda row: row.home_team_points if row.home_away == 'home_team' else row.away_team_points, axis=1)
matches_melt['goal'] = matches_melt.apply(lambda row: row.home_team_goal if row.home_away == 'home_team' else row.away_team_goal, axis=1)

# Teams by season (add 'home_away' to the groupby's to get at home/away stats)

teams_by_season = matches_melt.groupby(by=['country_name', 'league_name', 'season', 'team', ]).points.agg(['sum', 'count']).rename(columns={'sum': 'total_points', 'count': 'n_games'})
teams_by_season['perc_points'] = teams_by_season.total_points / (3 * teams_by_season.n_games)
teams_by_season[['wins', 'losses', 'draws']] = matches_melt.groupby(by=['country_name', 'league_name', 'season', 'team']).agg({'win': 'sum', 'loss': 'sum', 'draw': 'sum'})
teams_by_season[['perc_wins', 'perc_losses', 'perc_draws']] = teams_by_season[['wins', 'losses', 'draws']].div(teams_by_season.n_games, axis=0)

# Matchups

matchups = matches_melt.groupby(by=['country_name', 'league_name', 'team', 'other_team']).agg({'win': ['count', 'sum'], 'loss': 'sum', 'draw': 'sum'})
matchups.columns = ['n_games', 'wins', 'losses', 'draws']
matchups[['perc_wins', 'perc_losses', 'perc_draws']] = matchups[['wins', 'losses', 'draws']].div(matchups.n_games, axis=0)

In [168]:
display(teams_by_season)

display(matchups)

total_points  \
country_name league_name              season    team                                    
Belgium      Belgium Jupiler League   2008/2009 Beerschot AC                       42   
                                                Club Brugge KV                     59   
                                                FCV Dender EH                      35   
                                                KAA Gent                           59   
                                                KRC Genk                           50   
...                                                                               ...   
Switzerland  Switzerland Super League 2015/2016 FC Thun                            41   
                                                FC Vaduz                           36   
                                                FC Zürich                          34   
                                                Grasshopper Club Zürich            53   
                                                Lugano                             35   

                                                                         n_games  \
country_name league_name              season    team                               
Belgium      Belgium Jupiler League   2008/2009 Beerschot AC                  34   
                                                Club Brugge KV                34   
                                                FCV Dender EH                 34   
                                                KAA Gent                      34   
                                                KRC Genk                      34   
...                                                                          ...   
Switzerland  Switzerland Super League 2015/2016 FC Thun                       36   
                                                FC Vaduz                      36   
                                                FC Zürich                     36   
                                                Grasshopper Club Zürich       36   
                                                Lugano                        36   

                                                                         perc_points  \
country_name league_name              season    team                                   
Belgium      Belgium Jupiler League   2008/2009 Beerschot AC                0.411765   
                                                Club Brugge KV              0.578431   
                                                FCV Dender EH               0.343137   
                                                KAA Gent                    0.578431   
                                                KRC Genk                    0.490196   
...                                                                              ...   
Switzerland  Switzerland Super League 2015/2016 FC Thun                     0.379630   
                                                FC Vaduz                    0.333333   
                                                FC Zürich                   0.314815   
                                                Grasshopper Club Zürich     0.490741   
                                                Lugano                      0.324074   

                                                                         wins  \
country_name league_name              season    team                            
Belgium      Belgium Jupiler League   2008/2009 Beerschot AC               11   
                                                Club Brugge KV             18   
                                                FCV Dender EH               9   
                                                KAA Gent                   17   
                                                KRC Genk                   15   
...                                                                       ...   
Switzerland  Switzerland Supe

n_games  \
country_name league_name              team         other_team                         
Belgium      Belgium Jupiler League   Beerschot AC Club Brugge KV                10   
                                                   FCV Dender EH                  2   
                                                   KAA Gent                      10   
                                                   KAS Eupen                      2   
                                                   KRC Genk                      10   
...                                                                             ...   
Switzerland  Switzerland Super League Servette FC  FC St. Gallen                  4   
                                                   FC Thun                        8   
                                                   FC Zürich                      8   
                                                   Grasshopper Club Zürich        8   
                                                   Neuchâtel Xamax                2   

                                                                            wins  \
country_name league_name              team         other_team                      
Belgium      Belgium Jupiler League   Beerschot AC Club Brugge KV              3   
                                                   FCV Dender EH               1   
                                                   KAA Gent                    3   
                                                   KAS Eupen                   1   
                                                   KRC Genk                    3   
...                                                                          ...   
Switzerland  Switzerland Super League Servette FC  FC St. Gallen               0   
                                                   FC Thun                     1   
                                                   FC Zürich                   3   
                                                   Grasshopper Club Zürich     4   
                                                   Neuchâtel Xamax             1   

                                                                            losses  \
country_name league_name              team         other_team                        
Belgium      Belgium Jupiler League   Beerschot AC Club Brugge KV                6   
                                                   FCV Dender EH                 0   
                                                   KAA Gent                      2   
                                                   KAS Eupen                     0   
                                                   KRC Genk                      5   
...                                                                            ...   
Switzerland  Switzerland Super League Servette FC  FC St. Gallen                 3   
                                                   FC Thun                       5   
                                                   FC Zürich                     3   
                                                   Grasshopper Club Zürich       4   
                                                   Neuchâtel Xamax               0   

                                                                            draws  \
country_name league_name              team         other_team                       
Belgium      Belgium Jupiler League   Beerschot AC Club Brugge KV               1   
                                                   FCV Dender EH                1   
                                                   KAA Gent                     5   
                                                   KAS Eupen                    1   
                                                   KRC Genk                     2   
...                                                                           ...   
Switzerland  Switzerland Super League Servette FC  F

### Old tables with ideas for other statistics

In [143]:
season_stats = pd.read_sql("""SELECT Country.name AS country_name, 
                                        League.name AS league_name, 
                                        season,
                                        count(distinct stage) AS number_of_stages,
                                        count(distinct HT.team_long_name) AS number_of_teams,
                                        avg(home_team_goal) AS avg_home_team_scors, 
                                        avg(away_team_goal) AS avg_away_team_goals, 
                                        avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                        avg(home_team_goal+away_team_goal) AS avg_goals, 
                                        sum(home_team_goal+away_team_goal) AS total_goals                                       
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                GROUP BY Country.name, League.name, season
                                HAVING count(distinct stage) > 10
                                ORDER BY Country.name, League.name, season DESC
                                ;""", conn)

matchups = pd.read_sql("""SELECT Country.name AS country_name, 
                                        League.name AS league_name, 
                                        HT.team_long_name AS  home_team,
                                        AT.team_long_name AS away_team,
                                        COUNT(*) AS games,
                                        COUNT(DISTINCT season) AS seasons,
                                        SUM(home_team_goal) AS home_team_goals, 
                                        SUM(away_team_goal) AS away_team_goal,
                                        SUM(CASE WHEN home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS home_wins,
                                        SUM(CASE WHEN home_team_goal = away_team_goal THEN 1 ELSE 0 END) AS draws,
                                        SUM(CASE WHEN home_team_goal < away_team_goal THEN 1 ELSE 0 END) AS laway_wins
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                GROUP BY
                                    home_team,
                                    away_team;""", conn)

In [145]:
matchups[matchups.home_team == "Real Madrid CF"].sort_values(by="seasons", ascending=False).head(10)

,country_name,league_name,home_team,away_team,games,seasons,home_team_goals,away_team_goal,home_wins,draws,laway_wins
4852,Spain,Spain LIGA BBVA,Real Madrid CF,Athletic Club de Bilbao,8,8,34,9,8,0,0
4859,Spain,Spain LIGA BBVA,Real Madrid CF,FC Barcelona,8,8,12,22,2,1,5
4881,Spain,Spain LIGA BBVA,Real Madrid CF,Valencia CF,8,8,13,7,4,4,0
4878,Spain,Spain LIGA BBVA,Real Madrid CF,Sevilla FC,8,8,27,11,7,0,1
4853,Spain,Spain LIGA BBVA,Real Madrid CF,Atlético Madrid,8,8,13,8,4,1,3
4864,Spain,Spain LIGA BBVA,Real Madrid CF,Málaga CF,8,8,25,7,6,2,0
4860,Spain,Spain LIGA BBVA,Real Madrid CF,Getafe CF,8,8,32,9,8,0,0
4868,Spain,Spain LIGA BBVA,Real Madrid CF,RCD Espanyol,8,8,27,5,6,2,0
4882,Spain,Spain LIGA BBVA,Real Madrid CF,Villarreal CF,7,7,22,7,6,1,0
4863,Spain,Spain LIGA BBVA,Real Madrid CF,Levante UD,6,6,19,3,6,0,0
